version 3.1

In [1]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader

# ✅ Set device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Client device: {device}")

# ✅ Load Client Dataset (Each Client Gets Different Data)
X_clients = np.load('X_clients.npy')
y_clients = np.load('y_clients.npy')

# ✅ Split Data into Two Clients
client_id = int(input("Enter Client ID (1 or 2): ")) - 1
X_client, y_client = np.array_split(X_clients, 2)[client_id], np.array_split(y_clients, 2)[client_id]

# ✅ Convert to Tensor & Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_client, dtype=torch.float32).to(device),
                              torch.tensor(y_client, dtype=torch.float32).to(device))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ✅ Define Local Model (Same as Global)
class LocalModel(nn.Module):
    def __init__(self, input_size):
        super(LocalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

input_size = X_client.shape[1]
model = LocalModel(input_size).to(device)

# ✅ Use FedProx Loss (Holds a Global Model for Regularization)
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.mu = mu

    def forward(self, preds, labels, local_params, global_params):
        base_loss = nn.BCELoss()(preds, labels)

        # ✅ Regularization Term (Difference Between Local & Global Parameters)
        prox_loss = sum((torch.norm(local_param - global_param) ** 2).sum()
                        for local_param, global_param in zip(local_params, global_params))

        return base_loss + (self.mu / 2) * prox_loss

# ✅ Define Optimizer, LR Scheduler & Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.85)
fedprox_loss = FedProxLoss(mu=0.01)  #

# ✅ Flower Client for Federated Learning
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader):
        self.model = model
        self.train_loader = train_loader
        self.global_params = None  

    # ✅ Get local model parameters to send to the server
    def get_parameters(self, config):
        return [val.cpu().detach().numpy() for val in self.model.parameters()]

    # ✅ Receive global model parameters & update local model
    def set_parameters(self, parameters):
        if isinstance(parameters, list):  # ✅ Ensure parameters are converted correctly
            params_ndarrays = [torch.tensor(p).to(device) for p in parameters]
        else:
            params_ndarrays = fl.common.parameters_to_ndarrays(parameters)

        self.global_params = params_ndarrays  # ✅ Store global params

        # ✅ Load parameters into the local model
        state_dict = self.model.state_dict()
        for name, param in zip(state_dict.keys(), self.global_params):
            state_dict[name] = param
        self.model.load_state_dict(state_dict)
        print("✅ Client: Parameters received & updated.")

    # ✅ Training (FedProx: Includes regularization using global parameters)
    def fit(self, parameters, config):
        self.set_parameters(parameters)  # ✅ Receive global model weights
        self.model.train()

        for epoch in range(10):  # ✅ Train for 10 epochs per round
            correct, total = 0, 0
            for X_batch, y_batch in self.train_loader:
                optimizer.zero_grad()
                y_pred = self.model(X_batch).squeeze()

                # ✅ Compute FedProx loss (Regularization to prevent deviation)
                loss = fedprox_loss(y_pred, y_batch, list(self.model.parameters()), self.global_params)
                loss.backward()
                optimizer.step()

                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)

        client_accuracy = correct / total
        print(f"📌 Client {client_id+1}: Training Completed | Accuracy: {client_accuracy:.4f}")

        scheduler.step()  # ✅ Adjust learning rate
        return self.get_parameters(config), total, {"accuracy": client_accuracy}

    # ✅ Validation (Local model accuracy)
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)  # ✅ Receive global model weights

        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in self.train_loader:
                y_pred = self.model(X_batch).squeeze()
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)

        val_accuracy = correct / total
        print(f"📌 Client {client_id+1}: Validation Accuracy: {val_accuracy:.4f}")
        return 0.0, total, {"accuracy": val_accuracy}

# ✅ Connect to Global Server (Using `start_client()`)
print(f"🚀 Client {client_id+1}: Connecting to the global server...")
fl.client.start_client(
    server_address="localhost:8080",
    client=FLClient(model, train_loader)  # ✅ No need for `.to_client()` in latest Flower versions
)


✅ Client device: cpu


FileNotFoundError: [Errno 2] No such file or directory: 'X_clients.npy'

version 3.3


In [2]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from phe import paillier  # Homomorphic Encryption (Paillier Cryptosystem)

# ✅ Initialize Homomorphic Encryption Keys
pub_key, priv_key = paillier.generate_paillier_keypair()

# ✅ Set device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Client device: {device}")

# ✅ Load Client Dataset
X_clients = np.load('X_clients.npy')
y_clients = np.load('y_clients.npy')

# ✅ Split Data into Clients
client_id = int(input("Enter Client ID (1 or 2): ")) - 1
X_client, y_client = np.array_split(X_clients, 2)[client_id], np.array_split(y_clients, 2)[client_id]

# ✅ Convert to Tensor & Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_client, dtype=torch.float32).to(device),
                              torch.tensor(y_client, dtype=torch.float32).to(device))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ✅ Define Local Model (Same as Global)
class LocalModel(nn.Module):
    def __init__(self, input_size):
        super(LocalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

input_size = X_client.shape[1]
model = LocalModel(input_size).to(device)

# ✅ FedProx Loss
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.mu = mu

    def forward(self, preds, labels, local_params, global_params):
        base_loss = nn.BCELoss()(preds, labels)

        # ✅ Regularization Term (Difference Between Local & Global Parameters)
        prox_loss = sum((torch.norm(local_param - global_param) ** 2).sum()
                        for local_param, global_param in zip(local_params, global_params))

        return base_loss + (self.mu / 2) * prox_loss

# ✅ Define Optimizer, LR Scheduler & Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.85)
fedprox_loss = FedProxLoss(mu=0.01)  # Mu controls global model influence

# ✅ Flower Client with Homomorphic Encryption
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, pub_key, priv_key):
        self.model = model
        self.train_loader = train_loader
        self.pub_key = pub_key
        self.priv_key = priv_key
        self.global_params = None  # ✅ Holds global model parameters

    # ✅ Encrypt Model Parameters
    def get_parameters(self, config):
        encrypted_params = []
        for param in self.model.parameters():
            param_np = param.cpu().detach().numpy().flatten()
            encrypted_param_list = [self.pub_key.encrypt(float(val)) for val in param_np]  # ✅ Encrypt each value
            encrypted_params.append(encrypted_param_list)
        return encrypted_params

    # ✅ Decrypt Received Global Parameters
    def set_parameters(self, encrypted_parameters):
        decrypted_params = []
        for param_list in encrypted_parameters:
            decrypted_param_list = [self.priv_key.decrypt(val) for val in param_list]  # ✅ Decrypt each value
            decrypted_params.append(torch.tensor(decrypted_param_list).to(device))

        # ✅ Load parameters into model (reshape to match original)
        state_dict = self.model.state_dict()
        for (name, param), decrypted_param in zip(state_dict.items(), decrypted_params):
            state_dict[name] = decrypted_param.view(param.shape)

        self.model.load_state_dict(state_dict)
        print("✅ Client: Parameters decrypted & updated.")

    # ✅ Training (FedProx: Includes regularization using global parameters)
    def fit(self, parameters, config):
        self.set_parameters(parameters)  # ✅ Receive global model weights
        self.model.train()

        for epoch in range(10):  # ✅ Train for 10 epochs per round
            correct, total = 0, 0
            for X_batch, y_batch in self.train_loader:
                optimizer.zero_grad()
                y_pred = self.model(X_batch).squeeze()

                # ✅ Compute FedProx loss (Regularization to prevent deviation)
                loss = fedprox_loss(y_pred, y_batch, list(self.model.parameters()), self.global_params)
                loss.backward()
                optimizer.step()

                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)

        client_accuracy = correct / total
        print(f"📌 Client {client_id+1}: Training Completed | Accuracy: {client_accuracy:.4f}")

        scheduler.step()  # ✅ Adjust learning rate
        return self.get_parameters(config), total, {"accuracy": client_accuracy}

    # ✅ Validation (Local model accuracy)
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)  # ✅ Receive global model weights

        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in self.train_loader:
                y_pred = self.model(X_batch).squeeze()
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)

        val_accuracy = correct / total
        print(f"📌 Client {client_id+1}: Validation Accuracy: {val_accuracy:.4f}")
        return 0.0, total, {"accuracy": val_accuracy}

# ✅ Connect to Global Server
print(f"🚀 Client {client_id+1}: Connecting to the global server...")
fl.client.start_client(
    server_address="localhost:8080",
    client=FLClient(model, train_loader, pub_key, priv_key)  # ✅ Fixed `TypeError`
)


✅ Client device: cpu


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


🚀 Client 1: Connecting to the global server...


INFO :      
INFO :      Received: train message 371392dc-2eb9-4df5-9ff6-4b1b12b4fbae
ERROR :     Client raised an exception.
Traceback (most recent call last):
  File "d:\Rreserch work\fedenvioremnt\.venv\lib\site-packages\flwr\client\app.py", line 570, in start_client_internal
    reply_message = client_app(message=message, context=context)
  File "d:\Rreserch work\fedenvioremnt\.venv\lib\site-packages\flwr\client\client_app.py", line 143, in __call__
    return self._call(message, context)
  File "d:\Rreserch work\fedenvioremnt\.venv\lib\site-packages\flwr\client\client_app.py", line 124, in ffn
    out_message = handle_legacy_message_from_msgtype(
  File "d:\Rreserch work\fedenvioremnt\.venv\lib\site-packages\flwr\client\message_handler\message_handler.py", line 128, in handle_legacy_message_from_msgtype
    fit_res = maybe_call_fit(
  File "d:\Rreserch work\fedenvioremnt\.venv\lib\site-packages\flwr\client\client.py", line 224, in maybe_call_fit
    return client.fit(fit_ins)
  Fi

TypeError: Expected encrypted_number to be an EncryptedNumber not: <class 'numpy.ndarray'>

3.4  inference attack simulation

In [4]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
import opacus

# ✅ Set device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Client device: {device}")

# ✅ Load Client Dataset (Each Client Gets Different Data)
X_clients = np.load('X_clients.npy')
y_clients = np.load('y_clients.npy')

# ✅ Split Data into Two Clients
client_id = int(input("Enter Client ID (1 or 2): ")) - 1
X_client, y_client = np.array_split(X_clients, 2)[client_id], np.array_split(y_clients, 2)[client_id]

# ✅ Convert to Tensor & Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_client, dtype=torch.float32).to(device),
                              torch.tensor(y_client, dtype=torch.float32).to(device))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ✅ Define Local Model (Same as Global)
class LocalModel(nn.Module):
    def __init__(self, input_size):
        super(LocalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

input_size = X_client.shape[1]
model = LocalModel(input_size).to(device)

# ✅ Use FedProx Loss (Holds a Global Model for Regularization)
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.mu = mu

    def forward(self, preds, labels, local_params, global_params):
        base_loss = nn.BCELoss()(preds, labels)
        prox_loss = sum((torch.norm(local_param - global_param) ** 2).sum()
                        for local_param, global_param in zip(local_params, global_params))
        return base_loss + (self.mu / 2) * prox_loss

# ✅ Define Optimizer, LR Scheduler & Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.85)
fedprox_loss = FedProxLoss(mu=0.01)

# ✅ Attach Differential Privacy Engine
privacy_engine = opacus.PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=0.5,  
    max_grad_norm=1.5
)

# ✅ Flower Client for Federated Learning
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader):
        self.model = model
        self.train_loader = train_loader
        self.global_params = None

    def get_parameters(self, config):
        return [val.cpu().detach().numpy() for val in self.model.parameters()]

    def set_parameters(self, parameters):
        if isinstance(parameters, list):
            params_ndarrays = [torch.tensor(p).to(device) for p in parameters]
        else:
            params_ndarrays = fl.common.parameters_to_ndarrays(parameters)
        self.global_params = params_ndarrays
        state_dict = self.model.state_dict()
        for name, param in zip(state_dict.keys(), self.global_params):
            state_dict[name] = param
        self.model.load_state_dict(state_dict)
        print("✅ Client: Parameters received & updated.")

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(5):
            correct, total = 0, 0
            for X_batch, y_batch in self.train_loader:
                optimizer.zero_grad()
                y_pred = self.model(X_batch).squeeze()
                loss = fedprox_loss(y_pred, y_batch, list(self.model.parameters()), self.global_params)
                loss.backward()
                optimizer.step()
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)
        client_accuracy = correct / total
        epsilon = privacy_engine.get_epsilon(1e-5)
        print(f"📌 Client {client_id+1}: Training Completed | Accuracy: {client_accuracy:.4f} | DP ε={epsilon:.2f}")
        scheduler.step()
        return self.get_parameters(config), total, {"accuracy": client_accuracy, "epsilon": epsilon}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in self.train_loader:
                y_pred = self.model(X_batch).squeeze()
                correct += ((y_pred > 0.5) == y_batch).sum().item()
                total += y_batch.size(0)
        val_accuracy = correct / total
        print(f"📌 Client {client_id+1}: Validation Accuracy: {val_accuracy:.4f}")
        return 0.0, total, {"accuracy": val_accuracy}

# ✅ Connect to Global Server
print(f"🚀 Client {client_id+1}: Connecting to the global server...")
fl.client.start_client(
    server_address="localhost:8080",
    client=FLClient(model, train_loader)
)

✅ Client device: cpu


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
05/09/2025 21:05:53:WARNING:DEPRECATED FEATURE: flwr.client.start_client() is deprecated.
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
05/09/2025 21:05:53:DEBUG:Opened insecure gRPC connection (no certificates were passed)
05/09/2025 21:05:53:DEBUG:ChannelConnectivity.IDLE
05/09/2025 21:05:53:DEB

🚀 Client 2: Connecting to the global server...
✅ Client: Parameters received & updated.


INFO :      Sent reply
05/09/2025 21:06:03:INFO:Sent reply
INFO :      
05/09/2025 21:06:03:INFO:
INFO :      Received: evaluate message e7c3724d-26a6-41ca-8b7b-7b7e1714c71c
05/09/2025 21:06:03:INFO:Received: evaluate message e7c3724d-26a6-41ca-8b7b-7b7e1714c71c
05/09/2025 21:06:03:WARNING:Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


📌 Client 2: Training Completed | Accuracy: 0.8514 | DP ε=11.45
✅ Client: Parameters received & updated.


...

📌 Client 2: Validation Accuracy: 0.8070
✅ Client: Parameters received & updated.


INFO :      Sent reply
05/09/2025 21:06:48:INFO:Sent reply


📌 Client 2: Training Completed | Accuracy: 0.8500 | DP ε=22.88


05/09/2025 21:08:21:DEBUG:gRPC channel closed


KeyboardInterrupt: 